# File for looking at text content

## Import Libraries:

In [5]:
import sklearn as sk
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from text_content import data

In [9]:
en_data = data[0]
print(en_data)

{'item_key': '000dd37d-a759-496a-97a5-11e9f2eddd6c', 'lang': 'en', 'content': "Hello there! Today, I want to inspire you to try some amazing sports equipment, namely a tennis racket, a rugby ball, and a bicycle.\n\nFirst up, let's talk about the tennis racket. The beauty of this sport is that it can be played no matter your age, gender, or skill level. When you hold a tennis racket, you're holding a tool that's capable of propelling a small ball at high speeds, making it incredible exercise for your mind and body. You'll develop hand-eye coordination, footwork, and the mental fortitude required to stay focused throughout a game.\n\nNext, we have the rugby ball. Rugby is a sport that's known for requiring physical strength, speed, and agility. The ball is oval in shape, which means it bounces in unpredictable ways, increasing the level of excitement and difficulty. By playing rugby, you'll gain a stronger sense of teamwork, resilience, and perseverance in the face of adversity. And who 

In [ ]:
sorted(people.items(), key=lambda item: item[1])